In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors


In [4]:


glove_file = 'glove.6B/glove.6B.100d.txt' 
word2vec_output_file = 'glove.6B.100d.word2vec.txt' 
glove2word2vec(glove_file, word2vec_output_file)


model_glove = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)





In [5]:
similar_words = model_glove.most_similar('australia')
print(similar_words)

[('zealand', 0.9042125940322876), ('england', 0.7858782410621643), ('australian', 0.7696145176887512), ('britain', 0.7670672535896301), ('canada', 0.7569618821144104), ('africa', 0.7522280812263489), ('scotland', 0.7264114618301392), ('wales', 0.7209109663963318), ('india', 0.7042096257209778), ('indies', 0.6983798146247864)]


so this works

In [6]:
df = pd.read_csv('SimLex-999.txt', sep = '\t')
df.head()

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93


In [7]:
l1 = df.iloc[1:, 0].tolist()
l2 = df.iloc[1:, 1].tolist()


In [8]:
l_combined = list(zip(l1, l2))


In [9]:
for i in range(len(l_combined)):
    word1, word2 = l_combined[i]
    if(word1 not in model_glove.key_to_index or word2 not in model_glove.key_to_index):
        l_combined.pop(i)


In [10]:
print(len(l_combined), len(l1))

998 998


all data is preseved, nothing is lost

In [11]:
import numpy as np
n = len(l1)
list_of_cosine_similarities = []
l1_t = [entry[0] for entry in l_combined]
l2_t = [entry[1] for entry in l_combined]
for i in range(n):
    word1_vec = model_glove[l1_t[i]]  # Getting the word vectors
    word2_vec = model_glove[l2_t[i]]  # Getting the word vectors
    cosine_similarity = np.dot(word1_vec, word2_vec) / (np.linalg.norm(word1_vec) * np.linalg.norm(word2_vec))
    list_of_cosine_similarities.append(cosine_similarity)

In [12]:
for i in range(len(l1_t)):
    print(list_of_cosine_similarities[i], l1_t[i], l2_t[i])

0.7552732 smart intelligent
0.78525466 hard difficult
0.5459936 happy cheerful
0.76676786 hard easy
0.5823263 fast rapid
0.78333604 happy glad
0.821938 short long
0.83606493 stupid dumb
0.82473576 weird strange
0.6349321 wide narrow
0.70591724 bad awful
0.76463455 easy difficult
0.7072134 bad terrible
0.615464 hard simple
0.61205405 smart dumb
0.59173715 insane crazy
0.4462506 happy mad
0.8087274 large huge
0.78765094 hard tough
0.52368957 new fresh
0.42561817 sharp dull
0.5612247 quick rapid
0.696346 dumb foolish
0.8290025 wonderful terrific
0.8156487 strange odd
0.5397665 happy angry
0.68565875 narrow broad
0.7649386 simple easy
0.39810023 old fresh
0.73728997 apparent obvious
0.8236533 inexpensive cheap
0.47458035 nice generous
0.31582147 weird normal
0.75502366 weird odd
0.3090168 bad immoral
0.6318903 sad funny
0.6992726 wonderful great
0.32966116 guilty ashamed
0.80805093 beautiful wonderful
0.6916094 confident sure
0.07970566 dumb dense
0.70820564 large big
0.28629345 nice cruel

In [14]:
rf = pd.read_csv('SimLex-999 2/refined.txt', sep = '\t')
rf.head()

,word1,word2,SimLex999
0,old,new,1.58
1,smart,intelligent,9.20
2,hard,difficult,8.77
3,happy,cheerful,9.55
4,hard,easy,0.95


In [15]:
pearson_val = model_glove.evaluate_word_pairs('SimLex-999 2/refined.txt')

In [16]:
print(pearson_val)

(PearsonRResult(statistic=0.3239589856507405, pvalue=8.103432214442202e-26), SignificanceResult(statistic=0.2961127514172582, pvalue=1.2008184645699354e-21), 0.10010010010010009)


In [17]:
model_glove.save('./glove_model.model')